## Importing Libraries

In [64]:
import pandas as pd
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime
from datetime import datetime
import geopandas
from sklearn.impute import SimpleImputer


## Getting to know the Data


### Description of the Provided CSV Files

- **ticket_data.csv**:
  Contains a ticket history where each row represents a ticket proposal on tictactrip.

- **cities.csv**:
  Contains information about cities served by tictactrip. Links can be established through the columns 'o_city' (origin_city) and 'd_city' (destination_city) in the ticket_data.

- **stations.csv**:
  Includes data about stations served by tictactrip. Links can be established through the columns 'o_station' and 'd_station' in the ticket_data.

- **providers.csv**:
  Provides information about different providers. Relationships can be made via the 'company' column in ticket_data. A provider refers to a subsidiary company. For instance, TGV and TER are two providers under VSC (voyages-sncf).

The provided information outlines the content and relationships between the datasets. These relationships are established through specific columns across the datasets, facilitating data linkage and analysis within the context of tictactrip services.


In [19]:
cities = pd.read_csv("Data\\cities.csv")
providers = pd.read_csv("Data\\providers.csv")
stations = pd.read_csv("Data\\stations.csv")
ticket = pd.read_csv("Data\\ticket_data.csv")

In [50]:
dataframes = [("cities", cities), ("providers", providers), ("stations", stations), ("ticket", ticket)]

for df_name, df in dataframes:

    print(f"Columns of {df_name} \n{df.columns.tolist()}\n")

Columns of cities 
['id', 'local_name', 'unique_name', 'latitude', 'longitude', 'population']

Columns of providers 
['id', 'company_id', 'provider_id', 'name', 'fullname', 'has_wifi', 'has_plug', 'has_adjustable_seats', 'has_bicycle', 'transport_type']

Columns of stations 
['id', 'unique_name', 'latitude', 'longitude']

Columns of ticket 
['id', 'company', 'o_station', 'd_station', 'departure_ts', 'arrival_ts', 'price_in_cents', 'search_ts', 'middle_stations', 'other_companies', 'o_city', 'd_city']



## Renaming Columns in Datasets for Enhanced Clarity and Manageability

In this section, columns within multiple datasets have been renamed utilizing Pandas functionality. The objective behind this action is to enhance the clarity and manageability of the datasets by assigning more explicit and descriptive column names.

Below is the Python code snippet showcasing the column renaming process:

In [51]:


cities.drop(['local_name'], axis=1, inplace=True)
cities.rename(columns={'id':'city_id',
                            'unique_name':'city_name',
                            'latitude':'city_latitude',
                            'longitude':'city_longitude',
                            'population':'city_population'}, inplace=True)


providers.rename(columns={'id':'true_company_id',
                               'name':'company_name',
                               'fullname':'company_fullname'}, inplace=True)

stations.rename(columns={'id':'station_id',
                              'unique_name' : 'station_name',
                              'latitude':'station_latitude',
                              'longitude':'station_longitude'}, inplace=True)


ticket.rename(columns={'id':'ticket_id'}, inplace=True)

     

## Exploring the Datasets

### Overview and Statistical Summary

The following code snippets offer insights into the datasets, providing a comprehensive understanding of its structure, statistical characteristics, and missing values.

### Cities

In [52]:
print(cities.shape)
cities.head()

(8040, 5)


,city_id,city_name,city_latitude,city_longitude,city_population
0,5159,padua,45.406435,11.876761,209678.0
1,76,barcelona,41.385064,2.173404,1611822.0
2,81,basel,47.593437,7.619812,NaN
3,259,erlangen,49.589674,11.011961,105412.0
4,11979,balș,44.353354,24.095672,NaN


In [53]:
cities.describe()

,city_id,city_latitude,city_longitude,city_population
count,8040.000000,8040.000000,8040.000000,3.690000e+02
mean,6328.786816,47.223456,6.299331,3.360184e+05
std,3504.698537,3.700934,7.747070,5.840409e+05
min,1.000000,1.370676,-101.397388,1.000460e+05
25%,2501.500000,44.662402,1.214282,1.253750e+05
50%,7003.500000,47.304750,4.887856,1.803020e+05
75%,9014.250000,49.611998,10.311597,3.098690e+05
max,12192.000000,64.145981,48.731938,8.416535e+06


In [54]:
cities.isnull().sum()

city_id               0
city_name             1
city_latitude         0
city_longitude        0
city_population    7671
dtype: int64

### Providers

In [55]:
print(providers.shape)
providers.head()

(227, 10)


,true_company_id,company_id,provider_id,company_name,company_fullname,has_wifi,has_plug,has_adjustable_seats,has_bicycle,transport_type
0,9,1,NaN,ouibus,Ouibus,True,True,True,False,bus
1,10,2,NaN,deinbus,Deinbus.de,False,False,False,False,bus
2,11,3,NaN,infobus,Infobus,False,False,False,False,bus
3,12,4,NaN,studentAgency,Student Agency,False,False,False,False,bus
4,13,5,NaN,flixbus,Flixbus,True,False,False,False,bus


In [56]:
providers.describe()

,true_company_id,company_id
count,227.000000,227.000000
mean,790.656388,9.343612
std,2251.823950,3.036065
min,9.000000,1.000000
25%,66.500000,9.000000
50%,127.000000,9.000000
75%,183.500000,9.000000
max,8389.000000,40.000000


In [57]:
providers.isnull().sum()

true_company_id          0
company_id               0
provider_id             14
company_name             0
company_fullname         0
has_wifi                 3
has_plug                 3
has_adjustable_seats     3
has_bicycle              3
transport_type           0
dtype: int64

### Stations

In [58]:
print(stations.shape)
stations.head()


(11035, 4)


,station_id,station_name,station_latitude,station_longitude
0,1,Aalen (Stuttgarter Straße),48.835296,10.092956
1,2,Aéroport Bordeaux-Mérignac,44.830226,-0.700883
2,3,Aéroport CDG,49.009900,2.559310
3,4,Aéroport de Berlin-Schönefeld,52.389446,13.520345
4,5,Aéroport de Dresden,51.123604,13.764737


In [59]:
stations.describe()

,station_id,station_latitude,station_longitude
count,11035.000000,11035.000000,11035.000000
mean,5518.026914,47.526678,7.580807
std,3185.719813,3.953972,63.655681
min,1.000000,-22.355790,-22.625538
25%,2759.500000,44.896447,1.358655
50%,5518.000000,47.759670,5.824010
75%,8276.500000,50.349956,12.166133
max,11036.000000,64.137571,6645.000000


In [60]:
stations.isnull().sum()

station_id           0
station_name         0
station_latitude     0
station_longitude    0
dtype: int64

### Tickets

In [61]:
print(ticket.shape)
ticket.head()

(74168, 12)


,ticket_id,company,o_station,d_station,departure_ts,arrival_ts,price_in_cents,search_ts,middle_stations,other_companies,o_city,d_city
0,6795025,8385,NaN,NaN,2017-10-13 14:00:00+00,2017-10-13 20:10:00+00,4550,2017-10-01 00:13:31.327+00,NaN,NaN,611,542
1,6795026,9,63.0,1044.0,2017-10-13 13:05:00+00,2017-10-14 06:55:00+00,1450,2017-10-01 00:13:35.773+00,"{149,418}",{13},611,542
2,6795027,8377,5905.0,6495.0,2017-10-13 13:27:00+00,2017-10-14 21:24:00+00,7400,2017-10-01 00:13:40.212+00,"{798,798,6794,6246}","{8377,8376}",611,542
3,6795028,8377,5905.0,6495.0,2017-10-13 13:27:00+00,2017-10-14 11:02:00+00,13500,2017-10-01 00:13:40.213+00,"{798,798,6794,6246}","{8377,8376}",611,542
4,6795029,8381,5905.0,6495.0,2017-10-13 21:46:00+00,2017-10-14 19:32:00+00,7710,2017-10-01 00:13:40.213+00,"{5983,5983}",{8380},611,542


In [62]:
ticket.describe()

,ticket_id,company,o_station,d_station,price_in_cents,o_city,d_city
count,7.416800e+04,74168.000000,32727.000000,32727.000000,74168.000000,74168.000000,74168.000000
mean,6.832108e+06,7109.565527,2907.132673,2347.858129,4382.711061,849.186105,883.776265
std,2.141060e+04,3005.380792,3347.629180,3090.798007,3739.325367,1485.791782,1654.698575
min,6.795025e+06,9.000000,3.000000,3.000000,300.000000,5.000000,1.000000
25%,6.813567e+06,8376.000000,400.000000,396.000000,1900.000000,485.000000,453.000000
50%,6.832108e+06,8385.000000,701.000000,575.000000,3350.000000,628.000000,562.000000
75%,6.850650e+06,8385.000000,6246.000000,4538.000000,5250.000000,628.000000,628.000000
max,6.869192e+06,8387.000000,11017.000000,11017.000000,38550.000000,12190.000000,12190.000000


In [63]:
ticket.isnull().sum()

ticket_id              0
company                0
o_station          41441
d_station          41441
departure_ts           0
arrival_ts             0
price_in_cents         0
search_ts              0
middle_stations    41441
other_companies    41441
o_city                 0
d_city                 0
dtype: int64

## Handling Missing Values Using SimpleImputer from Scikit-Learn

In this scenario, the `SimpleImputer` class from Scikit-Learn is employed to replace missing values with the most frequent data found within each column across multiple datasets. This technique helps in ensuring that missing values are substituted with the most commonly occurring values, thus preserving the integrity of the data.

The Python code snippet below demonstrates the application of `SimpleImputer` for each dataset:

In [65]:

my_imputer = SimpleImputer(strategy='most_frequent')

tickets = pd.DataFrame(my_imputer.fit_transform(ticket), columns=ticket.columns)
cities = pd.DataFrame(my_imputer.fit_transform(cities), columns=cities.columns)
stations = pd.DataFrame(my_imputer.fit_transform(stations), columns=stations.columns)
providers = pd.DataFrame(my_imputer.fit_transform(providers), columns=providers.columns)
     

##  Calculating the Prices

In [77]:
# Minimum, average, and maximum prices
min_price = ticket['price_in_cents'].min()
max_price = ticket['price_in_cents'].max()
mean_price = ticket['price_in_cents'].mean()

print(f"Price Information:")
print(f"Minimum Price: {min_price} cents --> {min_price/100} euros ")
print(f"Maximum Price: {max_price} cents --> {max_price/100} euros ")
print(f"Average Price: {mean_price:.2f} cents --> {(mean_price/100):.2f} euros \n")

Price Information:
Minimum Price: 300 cents --> 3.0 euros 
Maximum Price: 38550 cents --> 385.5 euros 
Average Price: 4382.71 cents --> 43.83 euros 



In [85]:
# more details in cents : LES PERCENTILES
Q1_25e_percentile=ticket.price_in_cents.describe()['25%']
Median=ticket.price_in_cents.describe()['50%']
Q3_75e_percentile=ticket.price_in_cents.describe()['75%']
print(Q1_25e_percentile/100)
print(Median/100)
print(Q3_75e_percentile/100)

19.0
33.5
52.5
